In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

from pathlib import Path

from attila.data.parse import parse_data, get_data
from attila.util.config import get_config
from attila.util.plots import plot_sample
from attila.experiments.data import load_experiments
from attila.experiments.do import do_batch_experiments
from attila.experiments.tools import create_tex_experiments

In [3]:
_here = Path('.').resolve()
config = get_config(_here / 'config.ini')

data_path = _here / config.get('data', 'folder')
data_path = data_path.resolve()

out_path = Path(config.get('experiments', 'output folder')).resolve()
out_path.mkdir(parents=True, exist_ok=True)  # rm and mkdir if existing

images_path = data_path / config.get('data', 'images')
masks_path = data_path / config.get('data', 'masks')

In [4]:
raw = get_data(images_path, masks_path)
X, y = parse_data(
    raw,
    (config.getint('image', 'width'), config.getint('image', 'height'))
)

In [5]:
experiments_file = _here / 'experiments.json'
experiments = load_experiments(experiments_file)

In [ ]:
do_batch_experiments(experiments, (X, y), config, out_path)

testing data: X ~ (28, 240, 240, 1), y ~ (28, 240, 240, 3)
ready to perform #1 / 3 batch of experiments
augmented training data: X ~ (216, 240, 240, 1), y ~ (216, 240, 240, 3)
ready to perform 8 experiments
=== experiment #1 / 8: with_same
= dataset training: X ~ (216, 240, 240, 1), y ~ (216, 240, 240, 3)
= dataset validation: X ~ (10, 240, 240, 1), y ~ (10, 240, 240, 3)
= dataset test: X ~ (28, 240, 240, 1), y ~ (28, 240, 240, 3)
=== model
= # layers: 35
= # total params: 7781891
= # trainable params: 7781891
= # non-trainable params: 0
Epoch 1/10
54/54 [==============================] - ETA: 0s - loss: 0.1024 - accuracy: 0.9688 - batch_metric-mean_IoU: 0.4621 - batch_metric-mean_DSC: 0.3730
Epoch 00001: loss improved from inf to 0.10239, saving model to /home/h4/stfo194b/scratch/attila/out/run-0/with_same/model.h5
54/54 [==============================] - 5s 85ms/step - loss: 0.1024 - accuracy: 0.9688 - batch_metric-mean_IoU: 0.4621 - batch_metric-mean_DSC: 0.3730 - val_loss: 0.0261 -

Epoch 4/10
53/54 [============================>.] - ETA: 0s - loss: 0.0140 - accuracy: 0.9909 - batch_metric-mean_IoU: 0.6715 - batch_metric-mean_DSC: 0.5165
Epoch 00004: loss improved from 0.01922 to 0.01394, saving model to /home/h4/stfo194b/scratch/attila/out/run-0/without_same/model.h5
54/54 [==============================] - 3s 55ms/step - loss: 0.0139 - accuracy: 0.9910 - batch_metric-mean_IoU: 0.6718 - batch_metric-mean_DSC: 0.5168 - val_loss: 0.0097 - val_accuracy: 0.9937 - val_batch_metric-mean_IoU: 0.6877 - val_batch_metric-mean_DSC: 0.5322 - lr: 0.0010
Epoch 5/10
53/54 [============================>.] - ETA: 0s - loss: 0.0117 - accuracy: 0.9923 - batch_metric-mean_IoU: 0.6950 - batch_metric-mean_DSC: 0.5337
Epoch 00005: loss improved from 0.01394 to 0.01161, saving model to /home/h4/stfo194b/scratch/attila/out/run-0/without_same/model.h5
54/54 [==============================] - 3s 53ms/step - loss: 0.0116 - accuracy: 0.9923 - batch_metric-mean_IoU: 0.6956 - batch_metric-mean

54/54 [==============================] - 2s 42ms/step - loss: 0.0111 - accuracy: 0.9928 - batch_metric-mean_IoU: 0.6611 - batch_metric-mean_DSC: 0.5480 - val_loss: 0.0131 - val_accuracy: 0.9910 - val_batch_metric-mean_IoU: 0.6379 - val_batch_metric-mean_DSC: 0.4929 - lr: 0.0010
Epoch 8/10
53/54 [============================>.] - ETA: 0s - loss: 0.0132 - accuracy: 0.9915 - batch_metric-mean_IoU: 0.6417 - batch_metric-mean_DSC: 0.5317
Epoch 00008: loss did not improve from 0.01108
54/54 [==============================] - 2s 36ms/step - loss: 0.0130 - accuracy: 0.9915 - batch_metric-mean_IoU: 0.6431 - batch_metric-mean_DSC: 0.5321 - val_loss: 0.0075 - val_accuracy: 0.9947 - val_batch_metric-mean_IoU: 0.6943 - val_batch_metric-mean_DSC: 0.5331 - lr: 0.0010
Epoch 9/10
53/54 [============================>.] - ETA: 0s - loss: 0.0093 - accuracy: 0.9942 - batch_metric-mean_IoU: 0.6777 - batch_metric-mean_DSC: 0.5695
Epoch 00009: loss improved from 0.01108 to 0.00925, saving model to /home/h4/st

In [ ]:
create_tex_experiments(config, out_path, out_path / 'tables.tex')